# Пайплайн предобработки данных
Воспроизведение в виде пайплайна всех действий по предобработке данных из ДЗ1.
Далее обученный пайплайн выгружается и используется в веб-сервисе для инференса (совместно с пайплайном модели).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

random.seed(42)
np.random.seed(42)

In [2]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_test.csv')

df_train_raw = df_train.copy()
df_test_raw = df_test.copy()

print("Train data shape:", df_train.shape)
print("Test data shape: ", df_test.shape)

Train data shape: (6999, 13)
Test data shape:  (1000, 13)


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


class DropDuplicates(BaseEstimator, TransformerMixin):
    '''Трансформер для удаления дубликатов в пайплайне'''
    def __init__(self, subset=None, keep='first'):
        self.subset = subset
        self.keep = keep

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.subset:
            return X.drop_duplicates(subset=self.subset, keep=self.keep)
        else:
            return X.drop_duplicates(keep=self.keep)


class DropColumns(BaseEstimator, TransformerMixin):
    '''Трансформер для удаления столбцов в пайплайне'''
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        data = X.copy()  # Создаем копию, т.к. менять исходный датафрейм нельзя
        return data.drop(labels=self.columns, axis=1)


class FirstWordExtractor(BaseEstimator, TransformerMixin):
    '''Трансформер для выделение первого слова и создания нового столбца в пайплайне.
    
    Пример: 
        make_extractor = MakeExtractor(['name'], ['make'])
        создать столбец 'make' (производитель) из столбца 'name' (название автомобиля)
    '''
    def __init__(self, source_cols: list[str], target_cols=None):
        self.source_cols = source_cols
        self.target_cols = target_cols

        if len(self.source_cols) != len(self.target_cols):
            raise ValueError("Количесвто элементов в 'source_cols' и 'target_cols' не совпадает.")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for source_col, target_col in zip(self.source_cols, self.target_cols):
            X[target_col] = X[source_col].str.split().str.get(0)
        return X


class FloatConverter(BaseEstimator, TransformerMixin):
    '''Трансформер для преобразования столбцов в числа в пайплайне'''
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
       for col in self.columns:
           try:
               X[col] = X[col].astype(float)
           except ValueError as e:
               print(f"Есть нечисловые значения в {col}: {e}. Заменены на NaN.")
               # после замены на NaN это можно заполнить медианами через SimpleImputer
               X[col] = pd.to_numeric(X[col], errors='coerce')
       return X


class IntConverter(BaseEstimator, TransformerMixin):
    '''Трансформер для преобразования столбцов в числа в пайплайне'''
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
       for col in self.columns:
           try:
               X[col] = X[col].astype(int)
           except ValueError as e:
               print(f"Есть нечисловые значения в {col}: {e}. Заменены на NaN.")
               # после замены на NaN это можно заполнить медианами через SimpleImputer
               X[col] = pd.to_numeric(X[col], downcast='integer',  errors='coerce')
       return X


class MedianImputer(BaseEstimator, TransformerMixin):
    '''Трансформер для заполнения медианой в пайплайне'''
    def __init__(self, columns):
        self.columns = columns
        self.medians = dict()
        self.is_fitted_ = False
        
    def fit(self, X, y=None):
        try:
            self.medians = X[self.columns].median().to_dict()
            self.is_fitted_ = True
        except KeyError as e:
            print(f"Не найдены столбцы {e} в датасете.")
        except ValueError as e:
            print(f"Невозможно вычислить медиану для столбца {e}.")
        return self

    def transform(self, X):
        if self.is_fitted_:
           for col in self.columns:
                X[col] = X[col].fillna(self.medians[col])
        return X


In [5]:
df_=df_train_raw

drop_duplicates = DropDuplicates(df_.columns.drop('selling_price').to_list())
drop_torque = DropColumns(['torque'])
make_make = FirstWordExtractor(
    source_cols=['name', 'mileage', 'engine', 'max_power'], 
    target_cols=['make', 'mileage', 'engine', 'max_power']
)
makeFloat = FloatConverter(['mileage', 'engine', 'max_power', 'seats'])
# imputMedians = SimpleImputer(strategy='median', missing_values=np.nan, )
medians = MedianImputer(['mileage', 'engine', 'max_power', 'seats'])
makeInt = IntConverter(['engine', 'seats'])


df_ = drop_duplicates.fit_transform(df_)
df_ = drop_torque.fit_transform(df_)
df_ = make_make.fit_transform(df_)
df_ = makeFloat.fit_transform(df_)
# df_[['mileage', 'engine', 'max_power', 'seats']] = imputMedians.fit_transform(df_[['mileage', 'engine', 'max_power', 'seats']])
df_ = medians.fit_transform(df_)
df_ = makeInt.fit_transform(df_)


Есть нечисловые значения в max_power: could not convert string to float: 'bhp'. Заменены на NaN.


In [6]:
df_.sample(2)
# df_.describe()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,make
5435,Mahindra Xylo D2 BS IV,2010,465000,77088,Diesel,Individual,Manual,First Owner,14.0,2489,95.00,8,Mahindra
177,Maruti Wagon R LXI CNG,2016,360000,50000,CNG,Individual,Manual,First Owner,26.6,998,58.16,5,Maruti


In [7]:
df_.shape

(5840, 13)

## Полный пайплайн предварительной обработки данных.

In [9]:
df_ = df_train_raw
df__ = df_test_raw

pipe_preprocess = Pipeline([
    # ('duplicates', DropDuplicates(df_.columns.drop('selling_price').to_list())),
    ('dropper', DropColumns(['torque'])),
    ('make_extractor', FirstWordExtractor(
        source_cols=['name', 'mileage', 'engine', 'max_power'], 
        target_cols=['make', 'mileage', 'engine', 'max_power'] )),
    ('float_converter', FloatConverter(['mileage', 'engine', 'max_power', 'seats'])),
    ('median', MedianImputer(['mileage', 'engine', 'max_power', 'seats'])),
    ('int_converter', IntConverter(['engine', 'seats'])),

])

pipe_preprocess.fit(df_)
# df_ = preprocess_pipe.transform(df_)
# df_.head(2)


Есть нечисловые значения в max_power: could not convert string to float: 'bhp'. Заменены на NaN.


Pipeline(steps=[('dropper', DropColumns(columns=['torque'])),
                ('make_extractor',
                 FirstWordExtractor(source_cols=['name', 'mileage', 'engine',
                                                 'max_power'],
                                    target_cols=['make', 'mileage', 'engine',
                                                 'max_power'])),
                ('float_converter',
                 FloatConverter(columns=['mileage', 'engine', 'max_power',
                                         'seats'])),
                ('median',
                 MedianImputer(columns=['mileage', 'engine', 'max_power',
                                        'seats'])),
                ('int_converter', IntConverter(columns=['engine', 'seats']))])

In [10]:

df_ = drop_duplicates.fit_transform(df_)
df_ = pipe_preprocess.transform(df_)
df_.sample(2)
# df_.describe()

Есть нечисловые значения в max_power: could not convert string to float: 'bhp'. Заменены на NaN.


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,make
4417,Ford Figo Aspire 1.2 Ti-VCT Trend,2018,590000,15000,Petrol,Individual,Manual,First Owner,18.16,1196,86.8,5,Ford
4524,Tata Tiago 1.2 Revotron XZ WO Alloy,2017,450000,30000,Petrol,Individual,Manual,First Owner,23.84,1199,84.0,5,Tata


In [11]:
df_.shape

(5840, 13)

In [12]:
df__ = pipe_preprocess.transform(df__)
df__.head(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,make
0,Mahindra Xylo E4 BS IV,2010,229999,168000,Diesel,Individual,Manual,First Owner,14.0,2498,112.0,7,Mahindra
1,Tata Nexon 1.5 Revotorq XE,2017,665000,25000,Diesel,Individual,Manual,First Owner,21.5,1497,108.5,5,Tata


In [13]:
df__.shape

(1000, 13)

In [14]:
import pickle

# сохраним весь пайплайн `pipe_preprocess`` в pickle
pickle.dump(pipe_preprocess, open('pipe_preprocess.pkl', 'wb'))

In [16]:
import dill as pickle
with open('pipe_preprocess2.pkl', 'wb') as f:
    pickle.dump(pipe_preprocess, f)